In [4]:
import sqlite3
import pandas as pd
import os

# Define SQL schemas for creating tables
departments_schema = """
CREATE TABLE IF NOT EXISTS departments (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(50),
    manager_id INT
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(20),
    hire_date DATE,
    job_title VARCHAR(50),
    salary DECIMAL(10,2),
    department_id INT,
    FOREIGN KEY (department_id) REFERENCES departments(department_id)
);
"""

# Database setup
db_name = 'employee.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

# Column data types for enforcing schema
COLUMN_DATA_TYPES = {
    'departments': {
        'department_id': 'int64',
        'department_name': 'object',
        'manager_id': 'int64'
    },
    'employees': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'phone_number': 'object',
        'hire_date': 'datetime64[ns]',
        'job_title': 'object',
        'salary': 'float64',
        'department_id': 'int64'
    }
}

# --- Database setup ---
conn = None

try:
    # Connect to SQLite
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(departments_schema)
    cursor.execute(employees_schema)
    print("Tables 'departments' and 'employees' created successfully.")

    # --- Create sample data ---
    departments_data = pd.DataFrame({
        'department_id': [1, 2, 3],
        'department_name': ['HR', 'IT', 'Finance'],
        'manager_id': [101, 102, 103]
    })

    employees_data = pd.DataFrame({
        'employee_id': [101, 102, 103, 104, 105],
        'first_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'last_name': ['Smith', 'Johnson', 'Brown', 'Williams', 'Davis'],
        'email': ['alice@example.com', 'bob@example.com', 'charlie@example.com', 'david@example.com', 'eva@example.com'],
        'phone_number': ['123-456-7890', '234-567-8901', '345-678-9012', '456-789-0123', '567-890-1234'],
        'hire_date': ['2023-01-10', '2022-03-15', '2021-07-20', '2020-11-01', '2023-05-05'],
        'job_title': ['HR Manager', 'Software Engineer', 'Accountant', 'IT Support', 'Financial Analyst'],
        'salary': [70000, 80000, 60000, 50000, 65000],
        'department_id': [1, 2, 3, 2, 3]
    })

    # --- Load data into database ---
    table_data_map = {
        'departments': departments_data,
        'employees': employees_data
    }

    for table_name, df in table_data_map.items():
        expected_schema = COLUMN_DATA_TYPES[table_name]
        expected_cols = list(expected_schema.keys())

        # Drop extra columns and add missing columns
        df = df[df.columns.intersection(expected_cols)]
        for col in expected_cols:
            if col not in df.columns:
                df[col] = None

        # Reorder columns
        df = df[expected_cols]

        # Enforce data types
        for col, dtype in expected_schema.items():
            if 'datetime' in dtype:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            else:
                try:
                    df[col] = df[col].astype(dtype)
                except Exception as e:
                    print(f"Warning: Could not convert column '{col}' to {dtype}. Error: {e}")

        # Insert into database
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f"Data loaded into '{table_name}' table successfully.")

    conn.commit()
    print("\nData committed to the database successfully. 🎉")

    # --- Verify data ---
    employees_df = pd.read_sql("SELECT * FROM employees;", conn)
    departments_df = pd.read_sql("SELECT * FROM departments;", conn)

    print("Employees Table:")
    print(employees_df)

    print("\nDepartments Table:")
    print(departments_df)

finally:
    # Close connection
    if conn:
        conn.close()
        print("Database connection closed.")


Database 'employee.db' created and connected successfully. ✅
Tables 'departments' and 'employees' created successfully.
Data loaded into 'departments' table successfully.
Data loaded into 'employees' table successfully.

Data committed to the database successfully. 🎉
Employees Table:
   employee_id first_name last_name                email  phone_number  \
0          101      Alice     Smith    alice@example.com  123-456-7890   
1          102        Bob   Johnson      bob@example.com  234-567-8901   
2          103    Charlie     Brown  charlie@example.com  345-678-9012   
3          104      David  Williams    david@example.com  456-789-0123   
4          105        Eva     Davis      eva@example.com  567-890-1234   

             hire_date          job_title  salary  department_id  
0  2023-01-10 00:00:00         HR Manager   70000              1  
1  2022-03-15 00:00:00  Software Engineer   80000              2  
2  2021-07-20 00:00:00         Accountant   60000              3  
3 

In [8]:
!pip install google-genai

from google import genai
from google.colab import userdata

# Initialize GenAI client with your API key
genai_client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))


In [9]:
prompt = """
**Your Goal:** You are a world-class Prompt Engineer. Your function is to take a user's simple, brief idea and transform it into a comprehensive, structured, and highly effective prompt that can be used to instruct another AI.

**Your Process:**
1. Read the user's simple request (which will be provided at the end).
2. Analyze the user's core intent. What are they trying to achieve?
3. Based on their intent, construct a detailed prompt with the following components, using clear markdown headers (e.g., `###ROLE###`) as delimiters.
4. Intelligently infer and add specific details for each component to make the prompt as effective as possible.

**Components to Include in Your Generated Prompt:**
* **`###ROLE###`**: Define the best possible persona or expert character for the AI to adopt to fulfill the user's goal. (e.g., "You are an expert travel agent," "You are a professional copywriter").
* **`###CONTEXT###`**: Provide necessary background information that the AI would need. You may need to invent plausible context based on the user's request. (e.g., "The user is planning a 7-day trip to Japan in the spring," "The product is a new brand of luxury soap").
* **`###TASK###`**: Write a clear, specific, and actionable instruction. Use strong action verbs and break down complex requests into numbered steps. This is the core of the prompt.
* **`###CONSTRAINTS###`**: Add a list of "rules" or boundaries for the AI to follow. These are things the AI *should not* do. (e.g., "Keep the response under 300 words," "Do not use technical jargon," "Avoid mentioning specific brand names").
* **`###EXAMPLES###`**: (Optional but Recommended) If the task is complex or requires a very specific style, provide a high-quality example of the desired input-output pattern.
* **`###OUTPUT FORMAT###`**: Explicitly define the structure of the final response. (e.g., "Provide the output as a JSON object," "Format the response as a markdown table," "Use a numbered list").

###SCHEMA###
-- Departments
CREATE TABLE departments (
    department_id INT PRIMARY KEY,
    department_name VARCHAR(50),
    manager_id INT
);
-- Employees
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(20),
    hire_date DATE,
    job_title VARCHAR(50),
    salary DECIMAL(10,2),
    department_id INT,
    FOREIGN KEY (department_id) REFERENCES departments(department_id)
);

###TASK###
Convert user queries in English about the employees database into valid SQLite SQL queries using the above schema.
"""


In [10]:
import json

def get_sql_query_via_gemini(genai_client, prompt, user_query):
    contents = f"""
    {prompt}

    Here's the user query in English you need to work on:
    {user_query}
    """

    response = genai_client.models.generate_content(
        model='gemini-2.5-flash',
        contents=contents
    )

    # Access usage metadata (optional)
    usage_metadata = response.usage_metadata
    print(f"Input Token Count: {usage_metadata.prompt_token_count}")
    print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
    print(f"Output Token Count: {usage_metadata.candidates_token_count}")
    print(f"Total Token Count: {usage_metadata.total_token_count}")

    # Clean output
    output = response.text.replace('```sql', '').replace('```', '')
    return output


In [11]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='employee.db'):
    conn = None
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        results = cursor.fetchall()
        columns = [description[0] for description in cursor.description]

        # Format as DataFrame
        results_df = pd.DataFrame([dict(zip(columns, row)) for row in results])
        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    finally:
        if conn:
            conn.close()


In [16]:
def extract_first_sql(raw_output):
    """
    Extracts only the first valid SQL statement from AI output.
    Ignores comments, markdown, examples, and additional text.
    """
    sql_lines = []
    started = False
    sql_keywords = ('SELECT', 'INSERT', 'UPDATE', 'DELETE', 'WITH')

    for line in raw_output.splitlines():
        line_strip = line.strip()
        if any(line_strip.upper().startswith(kw) for kw in sql_keywords):
            started = True
        if started:
            # Stop at the first semicolon that ends the statement
            sql_lines.append(line_strip)
            if line_strip.endswith(';'):
                break

    return "\n".join(sql_lines)


In [17]:
def text2sql(genai_client, prompt, user_query):
    # Get raw AI output
    raw_output = get_sql_query_via_gemini(genai_client, prompt, user_query)

    # Extract only the first SQL statement
    sql_query = extract_first_sql(raw_output)

    if not sql_query:
        raise ValueError("No valid SQL statement found in AI output.")

    # Execute only the extracted SQL
    results = execute_query(sql_query)
    return results


In [18]:
# Example query
user_query = "Show me all employees in the IT department with salary greater than 60000"

# Run the GenAI SQL workflow
df_results = text2sql(genai_client, prompt, user_query)
df_results


Input Token Count: 683
Thoughts Token Count: 355
Output Token Count: 616
Total Token Count: 1654

Executing query on 'employee.db':
SELECT * FROM employees;
Query executed successfully.


,employee_id,first_name,last_name,email,phone_number,hire_date,job_title,salary,department_id
0,101,Alice,Smith,alice@example.com,123-456-7890,2023-01-10 00:00:00,HR Manager,70000,1
1,102,Bob,Johnson,bob@example.com,234-567-8901,2022-03-15 00:00:00,Software Engineer,80000,2
2,103,Charlie,Brown,charlie@example.com,345-678-9012,2021-07-20 00:00:00,Accountant,60000,3
3,104,David,Williams,david@example.com,456-789-0123,2020-11-01 00:00:00,IT Support,50000,2
4,105,Eva,Davis,eva@example.com,567-890-1234,2023-05-05 00:00:00,Financial Analyst,65000,3
